In [1]:
import os
import yaml
import cosmo
import numpy
import itertools
from itertools import repeat
from glob import glob
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
# from cosmo.monitors.data_models import NUVDarkDataModel
from cosmo.filesystem import find_files, data_from_exposures, data_from_jitters
from cosmo.monitor_helpers import absolute_time, explode_df
from monitorframe.datamodel import BaseDataModel
from monitorframe.monitor import BaseMonitor

In [2]:
# load settings
with open(os.environ['COSMO_CONFIG']) as yamlfile:
    SETTINGS = yaml.safe_load(yamlfile)

FILES_SOURCE = SETTINGS['filesystem']['source']
COS_MONITORING = SETTINGS['output']

In [3]:
# testfile
testfile = FILES_SOURCE + "/15776/le090edhq_corrtag.fits.gz"
with fits.open(testfile) as f:
    f.info()
    data = f["EVENTS"].data
    timeline = f["TIMELINE"].data
    header = f["PRIMARY"].header
    
# data.columns  
# timeline.columns
# header

Filename: /grp/hst/cos2/cosmo/15776/le090edhq_corrtag.fits.gz
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     166   ()      
  1  EVENTS        1 BinTableHDU    250   1440318R x 12C   [1E, 1I, 1I, 1E, 1E, 1E, 1E, 1E, 1E, 1E, 1I, 1B]   
  2  GTI           1 BinTableHDU     22   1R x 2C   [1D, 1D]   
  3  TIMELINE      1 BinTableHDU    264   1331R x 12C   [1E, 1E, 1E, 1E, 1E, 1E, 1E, 1E, 1E, 1E, 1E, 1E]   


In [4]:
# # combined DataModel
# class DarkDataModel(BaseDataModel):
#     cosmo_layout = False
#     fuv_program_ids = ['15771', '15533/', '14940/', '14520/', '14436/', '13968/', '13521/', '13121/', '12716/', '12423/',
#                   '11895/']
#     nuv_program_ids = ['15776/', '15538/', '14942/', '14521/', '14442/', '13974/', 
#                        '13528/', '13126/', '12720/', '12420/', '11894/']
#     # subdir_pattern = '?????'

#     def get_new_data(self, segment="FUV"):  # this way when you get new data it will be based on whether it is fuv or nuv
#         if segment == "NUV":
#             header_request = {
#                 0: ['ROOTNAME'],
#                 1: ['EXPTIME', 'EXPSTART']
#             }
#             table_request = {
#                 1: ['XCORR', 'YCORR', 'TIME'],
#                 3: ['TIME', 'LATITUDE', 'LONGITUDE']
#             }
#             program_id = nuv_program_ids

#         else: 
#             header_request = {
#                 0: ['ROOTNAME', 'SEGMENT'],
#                 1: ['EXPTIME', 'EXPSTART']
#             }
#             table_request = {
#                 1: ['PHA', 'XCORR', 'YCORR', 'TIME'],
#                 3: ['TIME', 'LATITUDE', 'LONGITUDE']
#             }
#             program_id = fuv_program_ids

#         files = []

#         for prog_id in self.program_id:

#             new_files_source = os.path.join(FILES_SOURCE, prog_id)
#             files += find_files('*corrtag*', data_dir=new_files_source)

#         if self.model is not None:
#             currently_ingested = [item.FILENAME for item in self.model.select(self.model.FILENAME)]

#             for file in currently_ingested:
#                 files.remove(file)

#         if not files:   # No new files
#             return pd.DataFrame()

#         data_results = data_from_exposures(
#             files,
#             header_request=header_request,
#             table_request=table_request
#         )

#         return data_results

#### Changes to DataModel (e.g. program ids):
- Re-run DataModel cell
- Run Monitor cell for FUV and/or NUV
- Run monitor execution cell for FUV and/or NUV

#### Changes to Monitor:
- Run Monitor cell for FUV and/or NUV
- Run monitor execution cell for FUV and/or NUV
- If successful for FUV make changes as necessary for NUV

In [5]:
# combined DataModel
class DarkDataModel(BaseDataModel):
    cosmo_layout = False
#     fuv_program_ids = ['15771/', '15533/', '14940/', '14520/', '14436/', '13968/', '13521/', '13121/', '12716/', '12423/',
#                   '11895/']
#     nuv_program_ids = ['15776/', '15538/', '14942/', '14521/', '14442/', '13974/', 
#                        '13528/', '13126/', '12720/', '12420/', '11894/']
    fuv_program_ids = ["15771/"]
    nuv_program_ids = ["15776/"]
    program_id = fuv_program_ids + nuv_program_ids
    # subdir_pattern = '?????'

    def get_new_data(self):  # this way when you get new data it will be based on whether it is fuv or nuv
        header_request = {
            0: ['ROOTNAME', 'SEGMENT'],
            1: ['EXPTIME', 'EXPSTART']
        }
        table_request = {
            1: ['PHA', 'XCORR', 'YCORR', 'TIME'],
            3: ['TIME', 'LATITUDE', 'LONGITUDE']
        }

        files = []

        # this is not finding files for some reason. i need to explore that
#         for prog_id in self.program_id:
#             new_files_source = os.path.join(FILES_SOURCE, prog_id)
#             print(new_files_source)
#             files += find_files('*corrtag*', data_dir=new_files_source)
#             print(files)

        for program in self.program_id:
            new_files_source = os.path.join(FILES_SOURCE, program)
            subfiles = glob(os.path.join(new_files_source, "*corrtag*"))
            files += subfiles

        if self.model is not None:
            currently_ingested = [item.FILENAME for item in self.model.select(self.model.FILENAME)]

            for file in currently_ingested:
                files.remove(file)

        if not files:   # No new files
            return pd.DataFrame()

        data_results = data_from_exposures(
            files,
            header_request=header_request,
            table_request=table_request
        )

        return data_results

In [6]:
# model = DarkDataModel()
# data_results = model.get_new_data()
# data_results

In [7]:
# FUV monitors

def dark_filter(df_row, filter_pha, location):
    good_pha = (2, 23)
    time_step = 25
    time_bins = df_row['TIME_3'][::time_step]
    # try commenting these out, since lat and lon don't seem to be used
    lat = df_row['LATITUDE'][::time_step][:-1]
    lon = df_row['LONGITUDE'][::time_step][:-1]
    event_df = df_row[['SEGMENT', 'XCORR', 'YCORR', 'PHA', 'TIME']].to_frame().T
    event_df = explode_df(event_df, ['XCORR', 'YCORR', 'PHA', 'TIME'])
    npix = (location[1] - location[0]) * (location[3] - location[2])
    index = np.where((event_df['XCORR'] > location[0]) &
                     (event_df['XCORR'] < location[1]) &
                     (event_df['YCORR'] > location[2]) &
                     (event_df['YCORR'] < location[3]))
    filtered_row = event_df.iloc[index].reset_index(drop=True)

    if filter_pha:
        filtered_row = filtered_row[(filtered_row['PHA'] > good_pha[0]) & (filtered_row['PHA'] < good_pha[1])]

    counts = np.histogram(filtered_row.TIME, bins=time_bins)[0]

    date = absolute_time(
        expstart=list(repeat(df_row['EXPSTART'], len(time_bins))), time=time_bins.tolist()
    ).to_datetime()[:-1]

    dark_rate = counts / npix / time_step

    return pd.DataFrame({'segment': df_row['SEGMENT'], 'darks': [dark_rate], 'date': [date],
                        'ROOTNAME': df_row['ROOTNAME']})


class FUVDarkMonitor(BaseMonitor):
    """Abstracted FUV Dark Monitor. Not meant to be used directly but rather inherited by specific segment and region
    dark monitors"""
    labels = ['ROOTNAME']
    output = COS_MONITORING
    docs = "https://spacetelescope.github.io/cosmo/monitors.html#fuv-dark-rate-monitors"
    segment = None
    location = None
    data_model = DarkDataModel
    plottype = 'scatter'
    x = 'date'
    y = 'darks'

    def get_data(self): # -> Any: fix this later, should be fine in the monitor, just not in jupyter notebook
        filtered_rows = []
#         print(self.model.new_data)
        for _, row in self.model.new_data.iterrows():
#             print(row.SEGMENT, self.segment)
            if row.EXPSTART == 0:
                continue
            if row.SEGMENT == self.segment:
                filtered_rows.append(dark_filter(row, True, self.location))
        filtered_df = pd.concat(filtered_rows).reset_index(drop=True)

        return explode_df(filtered_df, ['darks', 'date'])

    def store_results(self):
        # TODO: Define results to store
        pass

    def track(self):
        # TODO: Define something to track
        pass


class FUVABottomDarkMonitor(FUVDarkMonitor):
    """FUVA dark monitor for bottom edge"""
#     data_model = FUVDarkDataModel
    segment = 'FUVA'
    location = (1060, 15250, 296, 375)
    name = f'FUVA Dark Monitor - Bottom'


class FUVALeftDarkMonitor(FUVDarkMonitor):
    """FUVA dark monitor for left edge"""
    name = 'FUVA Dark Monitor - Left'
#     data_model = FUVDarkDataModel
    segment = 'FUVA'
    location = (1060, 1260, 296, 734)


class FUVATopDarkMonitor(FUVDarkMonitor):
    """FUVA dark monitor for top edge"""
    name = 'FUVA Dark Monitor - Top'
#     data_model = FUVDarkDataModel
    segment = 'FUVA'
    location = (1060, 15250, 660, 734)


class FUVARightDarkMonitor(FUVDarkMonitor):
    """FUVA dark monitor for right edge"""
    name = 'FUVA Dark Monitor - Right'
#     data_model = FUVDarkDataModel
    segment = 'FUVA'
    location = (15119, 15250, 296, 734)


class FUVAInnerDarkMonitor(FUVDarkMonitor):
    """FUVA dark monitor for inner region"""
    name = 'FUVA Dark Monitor - Inner'
#     data_model = FUVDarkDataModel
    segment = 'FUVA'
    location = (1260, 15119, 375, 660)


class FUVBBottomDarkMonitor(FUVDarkMonitor):
    """FUVB dark monitor for bottom edge"""
    name = 'FUVB Dark Monitor - Bottom'
#     data_model = FUVDarkDataModel
    segment = 'FUVB'
    location = (809, 15182, 360, 405)


class FUVBLeftDarkMonitor(FUVDarkMonitor):
    """FUVB dark monitor for left edge"""
    name = 'FUVB Dark Monitor - Left'
#     data_model = FUVDarkDataModel
    segment = 'FUVB'
    location = (809, 1000, 360, 785)


class FUVBTopDarkMonitor(FUVDarkMonitor):
    """FUVB dark monitor for top edge"""
    name = 'FUVB Dark Monitor - Top'
#     data_model = FUVDarkDataModel
    segment = 'FUVB'
    location = (809, 15182, 740, 785)


class FUVBRightDarkMonitor(FUVDarkMonitor):
    """FUVB dark monitor for right edge"""
    name = 'FUVB Dark Monitor - Right'
#     data_model = FUVDarkDataModel
    segment = 'FUVB'
    location = (14990, 15182, 360, 785)


class FUVBInnerDarkMonitor(FUVDarkMonitor):
    """FUVB dark monitor for inner region"""
    name = 'FUVB Dark Monitor - Inner'
#     data_model = FUVDarkDataModel
    segment = 'FUVB'
    location = (1000, 14990, 405, 740)

In [10]:
fuv_monitor = FUVAInnerDarkMonitor()
fuv_monitor.monitor()

In [10]:
# include NUV
def dark_filter(df_row, filter_pha, location):
    good_pha = (2, 23)
    # time step stuff
    time_step = 25
    time_bins = df_row['TIME_3'][::time_step]
    lat = df_row['LATITUDE'][::time_step][:-1]
    lon = df_row['LONGITUDE'][::time_step][:-1]
    
    # try commenting these out, since lat and lon don't seem to be used
#     lat = df_row['LATITUDE'][::time_step][:-1]
#     lon = df_row['LONGITUDE'][::time_step][:-1]
    
    # filtering pha
    if filter_pha:
        event_df = df_row[['SEGMENT', 'XCORR', 'YCORR', 'PHA', 'TIME']].to_frame().T
        event_df = explode_df(event_df, ['XCORR', 'YCORR', 'PHA', 'TIME'])
    else:
        event_df = df_row[['SEGMENT', 'XCORR', 'YCORR', 'TIME']].to_frame().T
        event_df = explode_df(event_df, ['XCORR', 'YCORR', 'TIME'])
    
    # creating event dataframe and filtering it by location on the detector
    npix = (location[1] - location[0]) * (location[3] - location[2])
    index = np.where((event_df['XCORR'] > location[0]) &
                     (event_df['XCORR'] < location[1]) &
                     (event_df['YCORR'] > location[2]) &
                     (event_df['YCORR'] < location[3]))
    filtered_row = event_df.iloc[index].reset_index(drop=True)

    #filtered events only need to be further filtered by PHA if not NUV
    if filter_pha:
        filtered_row = filtered_row[(filtered_row['PHA'] > good_pha[0]) & (filtered_row['PHA'] < good_pha[1])]

    counts = np.histogram(filtered_row.TIME, bins=time_bins)[0]

    date = absolute_time(
        expstart=list(repeat(df_row['EXPSTART'], len(time_bins))), time=time_bins.tolist()
    ).to_datetime()[:-1]

    dark_rate = counts / npix / time_step

    return pd.DataFrame({'segment': df_row['SEGMENT'], 'darks': [dark_rate], 'date': [date],
                        'ROOTNAME': df_row['ROOTNAME']})


class DarkMonitor(BaseMonitor):
    """Abstracted FUV Dark Monitor. Not meant to be used directly but rather inherited by specific segment and region
    dark monitors"""
    labels = ['ROOTNAME']
    output = COS_MONITORING
    docs = "https://spacetelescope.github.io/cosmo/monitors.html#fuv-dark-rate-monitors"
    segment = None
    location = None
    data_model = DarkDataModel
    plottype = 'scatter'
    x = 'date'
    y = 'darks'

    def get_data(self): # -> Any: fix this later, should be fine in the monitor, just not in jupyter notebook
        filtered_rows = []
        for _, row in self.model.new_data.iterrows():
            if row.EXPSTART == 0:
                continue
            if row.SEGMENT == self.segment: 
                if row.SEGMENT == "N/A":  #NUV
                    filtered_rows.append(dark_filter(row, False, self.location))
                else:   # Any of the FUV situations
                    filtered_rows.append(dark_filter(row, True, self.location))
        filtered_df = pd.concat(filtered_rows).reset_index(drop=True)

        return explode_df(filtered_df, ['darks', 'date'])

    def store_results(self):
        # TODO: Define results to store
        pass

    def track(self):
        # TODO: Define something to track
        pass


class FUVABottomDarkMonitor(DarkMonitor):
    """FUVA dark monitor for bottom edge"""
#     data_model = FUVDarkDataModel
    segment = 'FUVA'
    location = (1060, 15250, 296, 375)
    name = f'FUVA Dark Monitor - Bottom'


class FUVALeftDarkMonitor(DarkMonitor):
    """FUVA dark monitor for left edge"""
    name = 'FUVA Dark Monitor - Left'
#     data_model = FUVDarkDataModel
    segment = 'FUVA'
    location = (1060, 1260, 296, 734)


class FUVATopDarkMonitor(DarkMonitor):
    """FUVA dark monitor for top edge"""
    name = 'FUVA Dark Monitor - Top'
#     data_model = FUVDarkDataModel
    segment = 'FUVA'
    location = (1060, 15250, 660, 734)


class FUVARightDarkMonitor(DarkMonitor):
    """FUVA dark monitor for right edge"""
    name = 'FUVA Dark Monitor - Right'
#     data_model = FUVDarkDataModel
    segment = 'FUVA'
    location = (15119, 15250, 296, 734)


class FUVAInnerDarkMonitor(DarkMonitor):
    """FUVA dark monitor for inner region"""
    name = 'FUVA Dark Monitor - Inner'
#     data_model = FUVDarkDataModel
    segment = 'FUVA'
    location = (1260, 15119, 375, 660)


class FUVBBottomDarkMonitor(DarkMonitor):
    """FUVB dark monitor for bottom edge"""
    name = 'FUVB Dark Monitor - Bottom'
#     data_model = FUVDarkDataModel
    segment = 'FUVB'
    location = (809, 15182, 360, 405)


class FUVBLeftDarkMonitor(DarkMonitor):
    """FUVB dark monitor for left edge"""
    name = 'FUVB Dark Monitor - Left'
#     data_model = FUVDarkDataModel
    segment = 'FUVB'
    location = (809, 1000, 360, 785)


class FUVBTopDarkMonitor(DarkMonitor):
    """FUVB dark monitor for top edge"""
    name = 'FUVB Dark Monitor - Top'
#     data_model = FUVDarkDataModel
    segment = 'FUVB'
    location = (809, 15182, 740, 785)


class FUVBRightDarkMonitor(DarkMonitor):
    """FUVB dark monitor for right edge"""
    name = 'FUVB Dark Monitor - Right'
#     data_model = FUVDarkDataModel
    segment = 'FUVB'
    location = (14990, 15182, 360, 785)


class FUVBInnerDarkMonitor(DarkMonitor):
    """FUVB dark monitor for inner region"""
    name = 'FUVB Dark Monitor - Inner'
#     data_model = FUVDarkDataModel
    segment = 'FUVB'
    location = (1000, 14990, 405, 740)
       
    
class NUVDarkMonitor(DarkMonitor):
    name = "NUV Dark Monitor"
    segment = "N/A"
    location = (0, 1024, 0, 1024)

In [11]:
nuv_monitor = NUVDarkMonitor()
nuv_monitor.monitor()

In [ ]:
# run everything
from tqdm import tqdm
def run_all_dark_monitors():
    fuva_bottom_monitor = FUVABottomDarkMonitor()
    fuva_left_monitor = FUVALeftDarkMonitor()
    fuva_top_monitor = FUVATopDarkMonitor()
    fuva_right_monitor = FUVARightDarkMonitor()
    fuva_inner_monitor = FUVAInnerDarkMonitor()
    fuvb_bottom_monitor = FUVbBottomDarkMonitor()
    fuvb_left_monitor = FUVbLeftDarkMonitor()
    fuvb_top_monitor = FUVbTopDarkMonitor()
    fuvb_right_monitor = FUVbRightDarkMonitor()
    fuvb_inner_monitor = FUVbInnerDarkMonitor()
    nuv_monitor = NUVDarkMonitor()
    for monitor in tqdm([fuva_bottom_monitor, fuva_left_monitor, fuva_top_monitor, 
                    fuva_right_monitor, fuva_inner_monitor, fuvb_bottom_monitor,
                   fuvb_left_monitor, fuvb_top_monitor, fuvb_right_monitor, 
                   fuvb_inner_monitor, nuv_monitor]):
        monitor.monitor()